Not working properly in RStudio or Visual Studio

Resort to ipython and plain text editor

This notebook is inspired by the [2nd place solution of Mechanisms of Action (MoA)<br>
Prediction](https://www.kaggle.com/c/lish-moa/discussion/202256).

This Python 3 environment comes with many helpful analytics libraries installed<br>
It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python<br>
For example, here's several helpful packages to load

In [28]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

Input data files are available in the read-only "../input/" directory<br>
For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os<br>
for dirname, _, filenames in os.walk('/kaggle/input'):<br>
    for filename in filenames:<br>
        print(os.path.join(dirname, filename))

You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" <br>
You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')<br>
test = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')

In [30]:
dataset_train = pd.read_csv("data/pseudos2.csv")
# dataset_test = pd.read_csv("santander/test.csv")
dataset_train.head(3)

,ID_Code,target,Achimill,Achiptar,Acinarve,Agrolat,Agrocapi,Agrocurt,Agrostol,Agrovine,...,Vicisati,Vicisepi,Violcani,Violhirt,Viollute,Violpalu,Violrivi,Violrupe,Weismicr,Weisspp
0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


numerical_cols = [f'cont{i}' for i in range(1, 15)]

In [31]:
target_col = 'target'

for c in numerical_cols:<br>
    prep = StandardScaler()<br>
    train[c] = prep.fit_transform(train[[c]])<br>
    test[c] = prep.transform(test[[c]])

In [37]:
X_train = dataset_train.drop(['ID_Code', 'target'], axis=1)
y_train = dataset_train['target']
#X_test = dataset_test.drop('ID_code', axis=1)

In [ ]:
X_train.head(2)
#X_test.head(2)

Just for now, use X_test as validation. Sort out later

cv = KFold(n_splits=5, shuffle=True, random_state=7)<br>
<br>
y_preds = []<br>
models = []<br>
oof_train = np.zeros((len(X_train),))<br>
<br>
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):<br>
    X_tr = X_train.loc[train_index, :]<br>
    X_val = X_train.loc[valid_index, :]<br>
    y_tr = y_train.loc[train_index]<br>
    y_val = y_train.loc[valid_index]

In [38]:
model = keras.Sequential([
#   layers.BatchNormalization(),
#   layers.Dropout(0.1),
    layers.Dense(4096, activation='relu'),
    layers.Reshape((256, 16)),
#   layers.BatchNormalization(),
#   layers.Dropout(0.1),
    layers.Conv1D(filters=16, kernel_size=5, strides=1, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='linear'),
])

In [39]:
model.compile(
  optimizer='adam',
  loss='mse',
  metrics=[keras.metrics.RootMeanSquaredError()]
  )

early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

model.fit(
    X_train, y_train,
    #validation_data=(X_val, y_val),
    batch_size=32,
    epochs=5,
    validation_split = 0.2,
    callbacks=[early_stopping],
)


Epoch 1/5
50/50 [==============================] - 4s 52ms/step - loss: 0.5025 - root_mean_squared_error: 0.7089 - val_loss: 0.1906 - val_root_mean_squared_error: 0.4366
Epoch 2/5
50/50 [==============================] - 2s 37ms/step - loss: 0.0648 - root_mean_squared_error: 0.2546 - val_loss: 0.1280 - val_root_mean_squared_error: 0.3577
Epoch 3/5
50/50 [==============================] - 2s 39ms/step - loss: 0.0288 - root_mean_squared_error: 0.1696 - val_loss: 0.0842 - val_root_mean_squared_error: 0.2902
Epoch 4/5
50/50 [==============================] - 2s 38ms/step - loss: 0.0189 - root_mean_squared_error: 0.1373 - val_loss: 0.0861 - val_root_mean_squared_error: 0.2934
Epoch 5/5
50/50 [==============================] - 2s 44ms/step - loss: 0.0128 - root_mean_squared_error: 0.1133 - val_loss: 0.0809 - val_root_mean_squared_error: 0.2844


In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4096)              2191360   
                                                                 
 reshape (Reshape)           (None, 256, 16)           0         
                                                                 
 conv1d (Conv1D)             (None, 252, 16)           1296      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 126, 16)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2016)              0         
                                                                 
 dense_1 (Dense)             (None, 16)                32272     
                                                        

# Taken from keras implementation
See https://www.keep-current.dev/convolution-networks-on-tabular-data/

In [65]:
def create_cnn( num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate ):
    inp = tf.keras.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    #x = tf.keras.layers.WeightNormalization(Dense(4096))(x)
    x = tf.keras.layers.Reshape((256, 16))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)
    #x = WeightNormalization(Conv1D(filters = 16, kernel_size = 5, activation = swish, use_bias = False, padding = 'SAME'))(x)
    x = AveragePooling1D(pool_size = 2)(x)
    xs = x
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)
    #x = WeightNormalization(Conv1D(filters = 16, kernel_size = 3, activation = swish, use_bias = True, padding = 'SAME'))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)
    #x = WeightNormalization(Conv1D(filters = 16, kernel_size = 3, activation = swish, use_bias = True, padding = 'SAME'))(x)
    x = Multiply()([x, xs])
    x = MaxPool1D(pool_size = 4, strides = 2)(x)
    x = Flatten()(x)
    x = BatchNormalization()(x)
    x = Activation(swish)(x)
    x = Dense(num_labels)(x)
    out = Activation('sigmoid')(x)
    model = Model(inputs=inp, outputs=out)
    model.summary()
    return model

No real idea how you use the create_cnn class

In [67]:
import tensorflow as tf
#num_columns = 200
#inp = tf.keras.Input(shape=(num_columns,))
model = create_cnn(num_columns=10, num_labels=2, hidden_units=[64, 32], dropout_rates=[0.2, 0.2], label_smoothing=0.1, learning_rate=0.001)

#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing), metrics=[tf.keras.metrics.AUC()])

#history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)

ValueError: ignored